In [3]:
# from RealTimeTDTracker import MOT
from Utils import *
import pickle
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
with open(r'D:\Test\2022-04-03-12-04-44.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [7]:
Off_tracking_pool = b

In [8]:
def get_summary_file_TR(post_seq,key,start_frame,app_seq,T):
    temp = np.array(post_seq)
    temp = temp.reshape((temp.shape[0],temp.shape[1],temp.shape[2]))
    # n x 2 x 6
    temp_xy = temp[:,:,:2]
    # n x 2 x 2
    dis_est = np.sqrt((temp_xy[:,:,0]**2 + temp_xy[:,:,1]**2))
    # n x 2 
    speed_xy = temp[:,:,2:4] * 10 
    # n x 2 x 2
    speed = np.sqrt((speed_xy[:,:,0]**2 + speed_xy[:,:,1]**2))
    # n x 2
    xyz_0 = np.concatenate([temp_xy[:,0],np.zeros(len(temp_xy)).reshape(-1,1)],axis = 1)
    xyz_1 = np.concatenate([temp_xy[:,1],np.zeros(len(temp_xy)).reshape(-1,1)],axis = 1)
    xyz = (xyz_0 + xyz_1)/2
    LLH_est = convert_LLH(xyz,T)
    est = np.concatenate([xyz_0,dis_est[:,0].reshape(-1,1),speed_xy[:,0],speed[:,0].reshape(-1,1),LLH_est],axis = 1)
    # x,y,z,d,s_x,s_y,s,L,L,H
    timestp = []
    for i in range(len(temp)):
        f = i + start_frame + 1
        timestp.append('%06.0f'%f)
    timestp = np.array(timestp).reshape(-1,1)
    objid = (np.ones(len(temp)) * key).astype(int).reshape(-1,1)
    summary = np.concatenate([objid,timestp,est],axis = 1)
    # obj_id,ts,x,y,z,d,s_x,s_y,s,L,L,H
    summary = pd.DataFrame(summary,columns = column_names_TR_2o)

    emp = []
    for app in app_seq:
        if type(app) == int:
            emp_row = np.empty(len(app_col_names))
            emp_row[:] = np.nan
            emp.append(emp_row)
        else:
            emp.append(app.flatten())

    app_df = pd.DataFrame(emp,columns = app_col_names)
    app_df.Length = app_df.Length.max()

    return summary,app_df

In [9]:
def save_result(Off_tracking_pool,ref_LLH,ref_xyz,f_path):

    if len(Off_tracking_pool) == 0:
        print('No Trajs Here')
    else:
        # print('Generating Traj Files...')
        T = generate_T(ref_LLH,ref_xyz)
        sums = []
        app_dfs = []
        keys = []
        start_frame = []
        lengths = []
        for key in Off_tracking_pool:  
            sum_file,app_df = get_summary_file_TR(Off_tracking_pool[key].post_seq,
                                        key,Off_tracking_pool[key].start_frame,Off_tracking_pool[key].app_seq,T) 
            sums.append(sum_file)
            app_dfs.append(app_df)
            keys.append(key)
            start_frame.append(Off_tracking_pool[key].start_frame)   
            lengths.append(len(sum_file))   

        sums = pd.concat(sums)
        app_dfs = pd.concat(app_dfs)
        sums = sums.reset_index(drop=True).astype('float64')
        app_dfs = app_dfs.reset_index(drop=True).astype('float64')

        classifier = pickle.load(open('./Classifier/Classifier.sav', 'rb'))
        X_test = np.array(app_dfs.loc[:,['Point_Cnt','Height','Length','Area']])
        pred = classifier.predict(X_test)
        sums = pd.concat([sums,app_dfs,pd.DataFrame(pred.reshape(-1,1),columns=['Class'])],axis = 1)
        sums.to_csv(f_path,index = False)

In [10]:
ref_LLH,ref_xyz = pd.read_csv(r'D:\LiDAR_Data\MidTown\Cheney\Calibration\xyz_ref.csv'),pd.read_csv(r'D:\LiDAR_Data\MidTown\Cheney\Calibration\LLE_ref.csv')
ref_LLH,ref_xyz = np.array(ref_LLH),np.array(ref_xyz)

In [11]:
save_result(Off_tracking_pool,ref_LLH,ref_xyz)